In [ ]:
import torch
import transformers
import charactertokenizer

# model_dir = 'ai-forever/charllama-2.6B'
model_dir = 'ai-forever/charllama-1.3B'
model_dir = "/raid/home/rajivratn/hemant_rajivratn/last/src/trainllm/charllama-finetuned/"


tokenizer = charactertokenizer.CharacterTokenizer.from_pretrained(model_dir)
# print(tokenizer.get_vocab().keys())
model = transformers.AutoModelForCausalLM.from_pretrained(model_dir)
device = torch.device("cpu")
model.to(device)
model.train()

/raid/home/rajivratn/anaconda3/envs/langspeech/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyError: 'char_ords'

In [ ]:
model

In [6]:
input_texts = [
    "Hemant is my name",
    "Heeemmmannnnt iiiis mmmmy n naaame ee",
    "DRNOT TL KTCRNT ST RCT S C CDR C T DGT RM RTCETPCR RT T DC GESDT RESDE R DR GCECGCORGT EFLERC NCGKE EDES EDCD LTKT R DERT TS F DNSL GRKTSNG RKRDGCTRLS CG NSTGT RT CNTR LTERTEPRDRLSHS DEL O T RTS TN T GL ETGTGS ET GCG DLS T GD GRGSTLSKST TC T RDRETD T L TOLGSNLTREGTE SRTGR SRE CN GT NKR RLGR CG DT G RDG CP DESCTGRT CEDT SLTRS T G GLG G RG TP D LE VDV ET LRU CR CSE GTL T LT RTDTG CRTS R DTSC DT NE"
]

input_texts = ["".join(i.upper()) for i in input_texts ]
input_texts

for text in input_texts:
    # Poetry completion
    prompt =  text 
    print(prompt)
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    input_ids = inputs['input_ids']

    # Forward pass to get logits
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits  # shape: (batch_size, sequence_length, vocab_size)
        
    # Shift logits and labels for causal loss
    # Predicted logits are for tokens [0 ... n-2], targets are [1 ... n-1]
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()

    # Flatten the tensors for loss calculation
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1))

    print(f"Cross-entropy loss: {loss.item():.4f}")

HEMANT IS MY NAME
Cross-entropy loss: 3.6417
HEEEMMMANNNNT IIIIS MMMMY N NAAAME EE
Cross-entropy loss: 3.8645
DRNOT TL KTCRNT ST RCT S C CDR C T DGT RM RTCETPCR RT T DC GESDT RESDE R DR GCECGCORGT EFLERC NCGKE EDES EDCD LTKT R DERT TS F DNSL GRKTSNG RKRDGCTRLS CG NSTGT RT CNTR LTERTEPRDRLSHS DEL O T RTS TN T GL ETGTGS ET GCG DLS T GD GRGSTLSKST TC T RDRETD T L TOLGSNLTREGTE SRTGR SRE CN GT NKR RLGR CG DT G RDG CP DESCTGRT CEDT SLTRS T G GLG G RG TP D LE VDV ET LRU CR CSE GTL T LT RTDTG CRTS R DTSC DT NE
Cross-entropy loss: 2.9609
